#### Prompts generator experimental.

In [2]:
import sys
sys.path.append('../')

import mmgclip
from attrdict import AttrDict

from hydra import compose, initialize
from omegaconf import DictConfig, OmegaConf
from random import sample

# for auto reload when changes are made in the package
%load_ext autoreload
%autoreload 2

/local/abdel/mmg-clip/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/abdel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
with initialize(config_path="../configs"):
    cfg = compose(config_name="train_multi_class_clf")

print(OmegaConf.to_yaml(cfg.dataset))

name: ImageLabelDataset
config:
  enums_class: MassShapeLabels
  search_col: mass_shape
  generate_label_prompt_sentence: false
  generate_label_prompt_report: false
  base_dataset_path: /storage/Features/features/png_archive/2D_100micron/0/
  annotated_dataset_path: data/02_data_T_regions
  lists_dataset_path: data/02_data_lists/data/lists
eval:
  dataset:
    name: ImageLabelDataset
  enum_classes:
  - MassShapeLabels
  method:
  - classifier
  - zeroshot
  - ova
split:
  train_split_ratio: 0.7
  test_split_ratio: 0.5
template:
  prompt_template: ''
  label: []
  template_keys: []



In [4]:
# convert the config dict to a class object
try:
    cfg['dataset']['config']['lists_dataset_path'] = "../data/02_data_lists/data/lists"
    cfg['dataset']['config']['annotated_dataset_path'] = "../data/02_data_T_regions"

    
    mmgconfig = AttrDict(cfg)
    print(mmgconfig.dataset.config.lists_dataset_path)
except Exception as e:
    print("An error occurred:", e)

../data/02_data_lists/data/lists


Available templates can be displayed.

In [6]:
# mmgclip.available_prompts_templates()

Generating a single sentence for a specific label.

In [6]:
print(
    mmgclip.generate_label_prompt_sentence("lesion", "positive", n=2, template="GENERAL_TEMPLATE"),
)

['findings are suggestive of lesion', 'The presence of lesion is noted.']


In [7]:
print(
    mmgclip.generate_label_prompt_sentence("Ill defined", "has_single_margin", n=2, template="MASS_MARGIN"),
)
print(
    mmgclip.generate_label_prompt_sentence(["Ill defined", "Spiculated"], "has_many_margins", n=2, template="MASS_MARGIN"),
)

['Mass margin is Ill defined.', 'Has one mass margin Ill defined.']
['Has several mass margins, which are Ill defined, Spiculated.', 'Has more than one mass margin, Ill defined, Spiculated.']


In [8]:
print(
    mmgclip.generate_label_prompt_sentence("oval", "has_single_shape", n=2, template="MASS_SHAPE"),
)

['mass shape oval is seen.', 'Has single mass shape that is oval.']


In [9]:
# load the annotated label dataset
dataset_df = mmgclip.create_dataset_df(config=mmgconfig).sort_values('image_id').reset_index(drop=True)
dataset_df

,image_id,image_label,mass_margin,mass_shape,has_mass,image_path
0,p0200000202cl,0,"[Ill defined, -1, -1, -1, -1, -1, -1]","[Oval, -1, -1, -1, -1, -1, -1]",True,/storage/Features/features/png_archive/2D_100m...
1,p0200000202cr,0,"[-1, -1, -1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1]",True,/storage/Features/features/png_archive/2D_100m...
2,p0200000202ml,0,"[Ill defined, -1, -1, -1, -1, -1, -1]","[Oval, -1, -1, -1, -1, -1, -1]",True,/storage/Features/features/png_archive/2D_100m...
3,p0200000202mr,0,"[-1, -1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1]",True,/storage/Features/features/png_archive/2D_100m...
4,p0200000301cl,0,[-1],[-1],False,/storage/Features/features/png_archive/2D_100m...
...,...,...,...,...,...,...
6440,p0202057000mr,1,[-1],[-1],False,/storage/Features/features/png_archive/2D_100m...
6441,p0202061100cl,1,[Spiculated],[Irregular],True,/storage/Features/features/png_archive/2D_100m...
6442,p0202061100cr,1,[-1],[-1],False,/storage/Features/features/png_archive/2D_100m...
6443,p0202061100ml,1,[Spiculated],[Irregular],True,/storage/Features/features/png_archive/2D_100m...


In [51]:
dataset_df = mmgclip.generate_label_prompt_report(dataset_df, 'image_description')
dataset_df

,image_id,image_label,mass_margin,mass_shape,has_mass,image_path,image_description
0,p0200000202cl,0,"[Ill defined, -1, -1, -1, -1, -1, -1]","[Oval, -1, -1, -1, -1, -1, -1]",True,/storage/Features/features/png_archive/2D_100m...,Benign appearance. findings are suggestive of ...
1,p0200000202cr,0,"[-1, -1, -1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1]",True,/storage/Features/features/png_archive/2D_100m...,This is a benign mammogram. findings are sugge...
2,p0200000202ml,0,"[Ill defined, -1, -1, -1, -1, -1, -1]","[Oval, -1, -1, -1, -1, -1, -1]",True,/storage/Features/features/png_archive/2D_100m...,Benign case. findings are suggesting mass Mass...
3,p0200000202mr,0,"[-1, -1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1]",True,/storage/Features/features/png_archive/2D_100m...,This is a benign mammogram. findings are sugge...
4,p0200000301cl,0,[-1],[-1],False,/storage/Features/features/png_archive/2D_100m...,This is a benign finding. no mass Doesn't have...
...,...,...,...,...,...,...,...
6440,p0202057000mr,1,[-1],[-1],False,/storage/Features/features/png_archive/2D_100m...,Malignant evaluation. There is no mass. Mass s...
6441,p0202061100cl,1,[Spiculated],[Irregular],True,/storage/Features/features/png_archive/2D_100m...,This mammogram is malignant. mass is definite....
6442,p0202061100cr,1,[-1],[-1],False,/storage/Features/features/png_archive/2D_100m...,"Malignant evaluation. No visible, present evid..."
6443,p0202061100ml,1,[Spiculated],[Irregular],True,/storage/Features/features/png_archive/2D_100m...,This is a malignant finding. findings are sugg...


In [52]:
dataset_df.iloc[13]

image_id                                                 p0200000402cr
image_label                                                          0
mass_margin          [Circumscribed, Circumscribed, -1, -1, -1, -1,...
mass_shape                           [Oval, Round, -1, -1, -1, -1, -1]
has_mass                                                          True
image_path           /storage/Features/features/png_archive/2D_100m...
image_description    Probably benign. findings are suggestive of ma...
Name: 13, dtype: object

In [68]:
def generate_gtr_prompt_sentence(key, n=1, **kwargs):
    
    # REPORT = {
    #     # M_MARG    : Mass Margin
    #     # M_SHAPE   : Mass Shape
    #     # M_MALIG   : Mass Malignancy, note that calc can also be malignant
    #     # Separate with a comma between the mass shape/margin, and benign/malig classification
    #     "gtr_mass:True":[
    #         "The mass was characterized by {M_MARG} {M_SHAPE} on imaging, suggesting a potential {M_MALIG} etiology.",
    #         "The observed mass demonstrated {M_MARG} {M_SHAPE}, necessitating consideration of its {M_MALIG} characteristics.",
    #         "The mass exhibited {M_MARG} {M_SHAPE}, suggesting potential {M_MALIG} pathology.",
    #         "The mass noted {M_MARG} {M_SHAPE}, raising suspicion for {M_MALIG}.",
    #         "The mass demonstrated {M_MARG} {M_SHAPE}, prompting concern for underlying {M_MALIG}.",
    #         "The mass indicated {M_MARG} {M_SHAPE}, prompting concern for potential {M_MALIG}.",
    #         "The imaging revealed a mass with {M_MARG} {M_SHAPE}, warranting further investigation for {M_MALIG}.",
    #         "The mass displayed {M_MARG} {M_SHAPE}, suggestive of {M_MALIG} features upon imaging.",
    #         "The of the mass seen on imaging were {M_MARG} {M_SHAPE}, prompting concern for {M_MALIG}.",
    #         "The imaging findings revealed a mass with {M_MARG} {M_SHAPE}, indicating further evaluation for {M_MALIG}.",
    #         "The mass demonstrated {M_MARG} {M_SHAPE}, prompting further evaluation for {M_MALIG} features.",
    #         "The mass displayed {M_MARG} {M_SHAPE}, suggestive of a {M_MALIG} lesion.",
    #         "Imaging revealed a mass with {M_MARG} {M_SHAPE}, indicative of possible {M_MALIG}.",
    #         "The imaging findings showed a mass with {M_MARG} {M_SHAPE}, indicative of potential {M_MALIG}.",
    #         "The found mass appeared {M_MARG} {M_SHAPE}, raising suspicion for {M_MALIG} etiology.",
    #         "Imaging revealed a mass with {M_MARG} {M_SHAPE}, suggestive of {M_MALIG} pathology.",
    #         "The mass demonstrated {M_MARG} {M_SHAPE}, indicating a likely {M_MALIG} etiology.",
    #         "The present mass appeared {M_MARG} {M_SHAPE}, indicating potential {M_MALIG} characteristics.",
    #         "The observed mass shown {M_MARG} {M_SHAPE}, indicative of {M_MALIG} behavior.",
    #     ],
    #     "gtr_calc:True":[
    #         "The mammogram revealed calcifications {C_DIST}, suggesting potential {C_MALIG} pathology.",
    #         "Identified calcifications exhibit features indicative of {C_MALIG} {C_DIST}.",
    #         "The mammography report notes the presence of calcifications with concerning features, suggesting a potential for {C_MALIG} {C_DIST}.",
    #         "Calcifications observed raised suspicion for {C_MALIG} {C_DIST}.",
    #         "Reported calcifications display {C_MALIG} characteristics {C_DIST}.",
    #         "Calcifications identified present with suspicious features, warranting further evaluation for {C_MALIG} {C_DIST}.",
    #         "The calcifications visualized exhibit {C_MALIG} characteristics {C_DIST}.",
    #         "Calcifications noted suggesting a higher likelihood of {C_MALIG} {C_DIST}.",
    #         "Observed calcifications appear {C_MALIG} {C_DIST}.",
    #         "The calcifications identified present with {C_MALIG} characteristics {C_DIST}."
    #     ],
    #     "gtr_mass:True&gtr_calc:True":[
    #         "The mammography report highlights the presence of a {M_MARG} {M_SHAPE} accompanied by calcifications, indicative of {M_MALIG}.",
    #         "The mammography findings reveal masses with {M_MARG} {M_SHAPE} and calcifications, suggesting a low likelihood of {M_MALIG}.",
    #         "Masses identified exhibit {M_MARG} {M_SHAPE} and calcifications with concerning features, raising suspicion for {M_MALIG}.",
    #         "Observed masses demonstrate a {M_MARG} {M_SHAPE}, along with calcifications, indicating a likelihood of {M_MALIG}.",
    #         "The mammography report notes mass with {M_MARG} {M_SHAPE} and calcifications demonstrating suspicious features, suggesting {M_MALIG}.",
    #         "Detected masses display {M_MARG} {M_SHAPE} and associated calcifications, indicating a low probability of {M_MALIG}.",
    #         "Masses visualized exhibit {M_MARG} {M_SHAPE} and calcifications with concerning features, indicating {M_MALIG}."
    #     ],
    #     "suspicious": [ # note that those must be assigned birads 0 
    #         "Additional imaging or information is needed to make an assessment."
    #     ],
    #     "no_gtr": [ # we can assign birads 1, 2, or 3; assigned to birads 1
    #         "No finding is present in the imaging.",
    #         "Mammogram shows no evidence of any abnormalities.",
    #         "Mammogram shows no abnormal findings.",
    #         "Radiologist assessment reveals no evidence of abnormalities.",
    #         "Breast tissue appears unremarkable with no signs of pathology.",
    #         "No suspicious lesions or abnormalities are observed.",
    #         "Mammogram shows no significant findings."
    #     ],
    #     "row.labels['birads']:True":[ # this will come after malig or calc or both togather prompts, within the same sentence 
    #         "assigning BIRADS score of {B_SCORE} based on the findings.",
    #         "the mammography report assigns a BIRADS score of {B_SCORE} to guide further clinical decisions.",
    #         "this concludes assigning a BIRADS score of {B_SCORE}.",
    #         "a BIRADS score of {B_SCORE} is assigned to effectively communicate the mammography findings.",
    #         "therefor assigning BIRADS score of {B_SCORE} that serves an importance in the interpretation of mammography results.",
    #         "as a conclusion BIRADS score {B_SCORE} guides clinical decisions.",
    #         "thus BIRADS score {B_SCORE} communicates findings effectively.",
    #         "BIRADS score {B_SCORE} reflects radiologist's assessment.",
    #         "assigned BIRADS score {B_SCORE} for clinical management."
    #     ],
    #     "gtr_histology>0":[ # we use values > 0
    #         "Histological analysis confirmed the presence of {HISTOLOGY}.", # consistent with the suspicious findings observed on mammography
    #         "{HISTOLOGY} histology is reported.",
    #         "The histology examination revealed {HISTOLOGY}.",
    #         "Histology {HISTOLOGY} is noted.",
    #         "The histological findings are consistent with {HISTOLOGY}.",
    #         "Histological analysis reported {HISTOLOGY}.",
    #         "The histological features are {HISTOLOGY}."
    #     ],
    #     "gtr_is_architectural_distortion:True":[
    #         "The mammogram displayed architectural distortion, indicating possible disruption or retraction of breast tissue.",
    #         "Architectural distortion was noted on mammography, suggestive of underlying changes in breast tissue organization, necessitating further assessment.",
    #         "The presence of architectural distortion on the mammogram raised concern.",
    #         "Mammographic findings revealed architectural distortion.",
    #         "The observed architectural distortion on mammography warranted correlation with clinical findings.",
    #         "The presence of architectural distortion on mammography necessitated careful evaluation.",
    #         "Mammographic evaluation revealed diffuse architectural distortion, prompting consideration for comprehensive breast imaging.",
    #         "Architectural distortion observed on mammography may indicate localized breast tissue changes.",
    #         "Mammographic findings showed architectural distortion, suggesting alterations in breast tissue architecture.",
    #         "Architectural distortion identified on mammography may represent focal tissue changes.",
    #         "The observed architectural distortion on mammography warranted consideration.",
    #         "Mammographic evaluation revealed architectural distortion.",
    #         "Architectural distortion noted on mammography may indicate localized tissue abnormalities.",
    #         "The presence of architectural distortion on mammography prompted consideration.",
    #         "The presence of architectural distortion on mammography warranted further investigation.",
    #     ],
    #     "gtr_is_architectural_distortion:False":[
    #         "Mammography showed no evidence of architectural distortion.",
    #         "No architectural distortion was noted on mammography.",
    #         "Mammographic evaluation revealed no architectural distortion.",
    #         "No evidence of architectural distortion was observed on mammography.",
    #         "Mammography showed no architectural distortion.",
    #         "No architectural distortion was identified on mammography.",
    #         "Mammography revealed no evidence of architectural distortion.",
    #         "Mammography showed no architectural distortion, consistent with normal tissue appearance.",
    #         "Mammography showed no architectural distortion, suggesting preserved tissue organization.",
    #         "No evidence of architectural distortion was noted on mammography.",
    #     ]
    # }

    REPORT = {
        "combining": [
            "with",
            "accompanied by",
            "alongside a",
        ],
        "concluding": [
            ", indicative of",
            ", consistent with",
            ", warranting",
            ", suggesting a"
        ],
        "gtr_mass:True":[
            "The mammography reveals a {M_MALIG} mass {combining} {M_MARG} {M_SHAPE}.",
            "This mammography showcases {M_MALIG} mass {combining} {M_MARG} {M_SHAPE}.",
            "The report highlights {M_MALIG} mass {combining} {M_MARG} {M_SHAPE}.",
            "This mammography report indicates a {M_MALIG} mass {combining} {M_MARG} {M_SHAPE}.",
            "The imaging shows a {M_MALIG} mass {combining} {M_MARG} {M_SHAPE}.",
            "The mammography suggesting underlying {M_MALIG} mass {combining} {M_MARG} {M_SHAPE}."
        ],
        "row.labels['birads']:True":[ # this will come after malig or calc or both togather prompts, within the same sentence 
            "assigning BIRADS score of {B_SCORE} based on the findings.",
            "the mammography report assigns a BIRADS score of {B_SCORE} to guide further clinical decisions.",
            "this concludes assigning a BIRADS score of {B_SCORE}.",
            "a BIRADS score of {B_SCORE} is assigned to effectively communicate the mammography findings.",
            "therefor assigning BIRADS score of {B_SCORE} that serves an importance in the interpretation of mammography results.",
            "as a conclusion BIRADS score {B_SCORE} guides clinical decisions.",
            "thus BIRADS score {B_SCORE} communicates findings effectively.",
            "BIRADS score {B_SCORE} reflects radiologist's assessment.",
            "assigned BIRADS score {B_SCORE} for clinical management."
        ],
    }

    # prompts list
    prompts_report = [] 

    # if key == "no_gtr":
    #     prompts_random = sample(REPORT[key], n)
    #     prompts_report = prompts_random


    if key == 'gtr_mass:True' or key == "gtr_mass:True&gtr_calc:True":
        M_MARG = kwargs.get('M_MARG', 'unknown')
        M_SHAPE = kwargs.get('M_SHAPE', 'unknown')
        M_MALIG = kwargs.get('M_MALIG')

        prompts_random = sample(REPORT[key], n)
        prompt_combining = sample(REPORT["combining"], n)[0]
        prompt_replacement = "{M_MARG} {M_SHAPE}"

        for prompt in prompts_random:
            if M_MARG == "unknown":
                # if no margin passed, no need to add unknown label to the sentence
                prompt_replacement = prompt_replacement.replace("{M_MARG} ", "")
            else:
                # if margin passed, add margin info
                prompt_replacement = prompt_replacement.replace("{M_MARG}", f"{M_MARG} margins")

            if M_SHAPE == "unknown":
                # if no shape passed, no need to add shape information to the sentence
                prompt_replacement = prompt_replacement.replace("{M_SHAPE}", "")
            else:
                prompt_replacement = prompt_replacement.replace("{M_SHAPE}", f", and {M_SHAPE} shape" if M_MARG != "unknown" else f"{M_SHAPE} shape")

            if M_MARG == "unknown" and M_SHAPE == "unknown":
                # if both are unknown, split the sentence from the comma, and remove that part
                prompt = prompt.split('{combining}')[0].replace("{M_MALIG}", M_MALIG).capitalize()
            else:
                prompt = prompt.replace("{M_MALIG}", M_MALIG).replace("{M_MARG} {M_SHAPE}", prompt_replacement).replace("{combining}", prompt_combining)
            prompts_report.append(prompt)

    # if key == 'gtr_calc:True':
    #     C_MALIG = kwargs.get('C_MALIG')
    #     C_DIST = kwargs.get('C_DIST', 'unknown')

    #     prompts_random = sample(REPORT[key], n)
    #     prompt_replacement = " {C_DIST}"

    #     for prompt in prompts_random:
    #         if C_DIST == "unknown":
    #             prompt_replacement = prompt_replacement.replace(" {C_DIST}", "")
    #         else:
    #             prompt_replacement = prompt_replacement.replace(" {C_DIST}", f" with {C_DIST} distribution")

    #         prompt = prompt.replace("{C_MALIG}", C_MALIG).replace(" {C_DIST}", prompt_replacement)
            
    #         prompts_report.append(prompt)

    if 'birads' in key:
        B_SCORE = kwargs.get('B_SCORE')
        prompts_random = sample(REPORT[key], n)

        for prompt in prompts_random:
            prompt = prompt.replace("{B_SCORE}", B_SCORE)
            
            prompts_report.append(prompt)

    # if key == "gtr_histology>0":
    #     HISTOLOGY = kwargs.get('HISTOLOGY')
    #     prompts_random = sample(REPORT[key], n)

    #     for prompt in prompts_random:
    #         prompt = prompt.replace("{HISTOLOGY}", HISTOLOGY)
    #         prompts_report.append(prompt)
            
    # if 'gtr_is_architectural_distortion' in key:
    #     prompts_random = sample(REPORT[key], n)
    #     prompts_report = prompts_random


    return ' '.join(prompts_report)


print(generate_gtr_prompt_sentence(key = 'gtr_mass:True', n=1, M_MALIG="malignant",  M_MARG="ill defined", M_SHAPE="unknown")) #  M_SHAPE="oval"

The report highlights malignant mass alongside a ill defined margins .
